# Face Mask Detection
**Author: Ishani Kathuria**

A real time face mask detection program developed using Tensorflow object detection api. The results are visualized using OpenCV.

Dependencies  
`pip install tensorflow`  
`pip install tensorflow-object-detection-api`  
`pip install opencv-python`

In [1]:
import os
# model
import tensorflow as tf
# vizualization
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

import cv2
import numpy as np

## Setting up the paths

Storing all the paths to the different folders in variable so they don't have to be written again and again.

In [2]:
WORKPLACE_PATH = 'Tensorflow/workplace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKPLACE_PATH + '/annotations'
IMAGE_PATH = WORKPLACE_PATH + '/images'
MODEL_PATH = WORKPLACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKPLACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

## Generate Labels

Generating a label map for the 2 labels, **mask** and **noMask**.

In [3]:
labels = [
    {'name':'mask', 'id':1},
    {'name':'noMask', 'id':2},
]

In [4]:
# with open(os.path.join(ANNOTATION_PATH, 'label_map.pbtxt'), 'w') as f:
#     for label in labels:
#         f.write('item { \n')
#         f.write('\tname:\'{}\'\n'.format(label['name']))
#         f.write('\tid:{}\n'.format(label['id']))
#         f.write('}\n')
# print('Generated label map')

## Generate TF records

Generating TF records needed for the Tensorflow model.

In [5]:
# if os.name == 'posix':
#     !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
#     !python3 {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}
# else:
#     !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
#     !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

## Download TF models

Downloading Tensorflow models.

In [6]:
# !cd Tensorflow && git clone https://github.com/tensorflow/models

In [7]:
# !cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Setting up pipeline config

Setting up the pipeline.config file for face mask detection by chaging the following parameters.

In [8]:
# !cp {PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

**model.ssd.num_classes** = 2

**train_config.batch_size** = 4

**train_config.fine_tune_checkpoint** = 'Tensorflow/workplace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'

**train_config.fine_tune_checkpoint_type** = "detection"

**train_input_reader.label_map_path** = 'Tensorflow/workplace/annotations/label_map.pbtxt'

**train_input_reader.tf_record_input_reader.input_path[:]** = 'Tensorflow/workplace/annotations/train.record'

**eval_input_reader[0].label_map_path** = 'Tensorflow/workplace/annotations/label_map.pbtxt'

**eval_input_reader[0].tf_record_input_reader.input_path[:]** = 'Tensorflow/workplace/annotations/test.record'

## Training the model

Training the model for the number of steps needed.

In [9]:
num = 50000

In [10]:
print("--- for posix ---")
print(f"""python3 {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps={num}""")

print("\n--- else ---")
print(f"""python {APIMODEL_PATH}/research/object_detection/model_main_tf2.py --model_dir={MODEL_PATH}/{CUSTOM_MODEL_NAME} --pipeline_config_path={MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config --num_train_steps={num}""")

--- for posix ---
python3 Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workplace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workplace/models/my_ssd_mobnet/pipeline.config --num_train_steps=50000

--- else ---
python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workplace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workplace/models/my_ssd_mobnet/pipeline.config --num_train_steps=50000


## Vizualization

Vizualizing the real time output with OpenCV.

In [11]:
num = int(input("Enter latest checkpoint: "))

Enter latest checkpoint: 51


In [12]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model = detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, f'ckpt-{num}')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [13]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH + '/label_map.pbtxt')

In [15]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw = 1,
                min_score_thresh = .8,
                agnostic_mode = False)

    cv2.imshow('face mask detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Conclusion

The model was trained for 50,000 steps and was able to achieve an accuracy of:

**noMask = 99%**  
**mask = 99%**